# 220223 과제
diff = 0to1 - 1to0
mean(diff)

# library

In [1]:
#데이터 분석 라이브러리
import numpy as np
print("numpy version: {}". format(np.__version__))

import pandas as pd

#시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(mpl.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#딥러닝 라이브러리
import tensorflow as tf

#기타 라이브러리
import os
import random
import time

#경고 에러 무시
import warnings
warnings.filterwarnings('ignore')
print('-'*50)

#시간 확인을 용이하게 해주는 라이브러리
from tqdm import tqdm

#결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from varname import nameof

numpy version: 1.19.5
matplotlib version: 3.4.2
seaborn version: 0.11.1
--------------------------------------------------


# GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# CPU

In [3]:
#GPU 사용 설정, -1이면 CPU 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# all fit _ 기존 allfit과 다름. (기존 allfit 방식이 틀렸음.)
with all fit trained model <br>
allfit2_seed42-06-0.5519.hdf5

In [11]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

# 2) MODEL
with tf.device('/device:GPU:0'):
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('allfit2_seed42-06-0.5519.hdf5')
    
# 3) baseline
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 

    results = []    
    preds = lstm2.predict(x)
    preds_mean = np.mean(preds) # base preds

    results.append({'feature':'BASELINE','baseline':preds_mean})
#     tf.device('/device:GPU:0').close()

In [12]:
pd.Series(preds.flatten(), name = 'preds').to_csv('new_allfit_preds_base.csv', index = False)

In [13]:
results

[{'feature': 'BASELINE', 'baseline': 0.5553324}]

## all to 1

In [ ]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

####################################################################################################################

# 2) MODEL
with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('allfit2_seed42-06-0.5519.hdf5')

# 3) all_to_1
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = x[:,:,k].copy()

        # all_to_1
        x[:,:,k] = 1

        # all_to_0
    #     x[:,:,k] = 0

        # inverse
    #     x[:,:,k] = np.where(x[:,:,k]==1, 2, x[:,:,k])
    #     x[:,:,k] = np.where(x[:,:,k]==0, 1, x[:,:,k])
    #     x[:,:,k] = np.where(x[:,:,k]==2, 0, x[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(x)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

#         loss_bce = bce(y_test, preds2).numpy()
#         results.append({'feature':features[k],'mean_preds2':np.mean(preds2)})
        x[:,:,k] = save_col

Using TensorFlow backend.


 22%|██▏       | 883/4068 [37:57<2:18:36,  2.61s/it]

In [ ]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_all_to_1 = df1.copy()
all_to_1_all_preds2 = preds2_dic.copy()
df_all_to_1.columns = ['mean_preds2']
df_all_to_1.to_csv('new_allfit_all_to_1_mean_preds2.csv', index = False)

In [ ]:
for i,j in enumerate(all_to_1_all_preds2.keys()):
    if i == 0:
        ori = pd.Series(all_to_1_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(all_to_1_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기
ori.to_csv('new_allfit_all_to_1_all_preds2.csv', index=False)

## all to 0

In [ ]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

####################################################################################################################

# 2) MODEL
with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('allfit2_seed42-06-0.5519.hdf5')

# 3) all_to_1
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = x[:,:,k].copy()

        # all_to_1
#         x[:,:,k] = 1

        # all_to_0
        x[:,:,k] = 0

        # inverse
    #     x[:,:,k] = np.where(x[:,:,k]==1, 2, x[:,:,k])
    #     x[:,:,k] = np.where(x[:,:,k]==0, 1, x[:,:,k])
    #     x[:,:,k] = np.where(x[:,:,k]==2, 0, x[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(x)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

        x[:,:,k] = save_col

In [ ]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_all_to_0 = df1.copy()
all_to_0_all_preds2 = preds2_dic.copy()
df_all_to_0.columns = ['mean_preds2']
df_all_to_0.to_csv('new_allfit_all_to_0_mean_preds2.csv', index = False)

In [ ]:
for i,j in enumerate(all_to_0_all_preds2.keys()):
    if i == 0:
        ori = pd.Series(all_to_0_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(all_to_0_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기
ori.to_csv('new_allfit_all_to_0_all_preds2.csv', index=False)

## inverse

In [ ]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

####################################################################################################################

# 2) MODEL
with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
    from keras.models import load_model
    lstm2 = load_model('allfit2_seed42-06-0.5519.hdf5')

# 3) all_to_1
    # FEATURES
    a = pd.read_csv('/project/LSH/total_data_7727.csv')
    features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


    preds2_dic = {} # 그래프 만들 때 쓸 dic
    preds2_mean_lst = [] # 이거는 df 만들 때만 사용.

    for k in tqdm(range(len(features))):

        # REVERSE ALL FEATURE K
        save_col = x[:,:,k].copy()

        # all_to_1
#         x[:,:,k] = 1

        # all_to_0
#         x[:,:,k] = 0

        # inverse
        x[:,:,k] = np.where(x[:,:,k]==1, 2, x[:,:,k])
        x[:,:,k] = np.where(x[:,:,k]==0, 1, x[:,:,k])
        x[:,:,k] = np.where(x[:,:,k]==2, 0, x[:,:,k])

        # COMPUTE BCE WITH FEATURE K all_to_1, preds2_mean_list
        preds2 = lstm2.predict(x)
        preds2_dic[features[k]] = preds2 # 그래프 만들 때 쓸 dic
        preds2_mean_lst.append(np.mean(preds2)) # 이거는 df 만들 때만 사용.

        x[:,:,k] = save_col

In [ ]:
df1 = pd.DataFrame(preds2_mean_lst)

######################################################################
df_inverse = df1.copy()
inverse_all_preds2 = preds2_dic.copy()
df_inverse.columns = ['mean_preds2']
df_inverse.to_csv('new_allfit_inverse_mean_preds2.csv', index = False)

In [ ]:
for i,j in enumerate(inverse_all_preds2.keys()):
    if i == 0:
        ori = pd.Series(inverse_all_preds2[j].flatten(), name = j)
    
    else:
        tmp = pd.Series(inverse_all_preds2[j].flatten(), name = j)
        ori = pd.concat([ori, tmp], axis = 1)   # ori 계속 덮어쓰기
ori.to_csv('new_allfit_inverse_all_preds2.csv', index=False)

# top_bot_lst

In [4]:
rf_top10_lst = [51277, 50912, 51006, 63323026201, 50862, 50983, 51003, 51275, 904224461, 50882]
rf_bot10_lst = [55390012210, 536479301, 536376701, 536185097, 55390013910, 55390014301, 527155201, 2704001, 517420125, 575022530]

M1_entropy_top10_lst = [409176230, 51003, 487980125, 50862, 50813, 50868, 409606211, 338004902, 45006701, 55390007310]
M1_entropy_bot10_lst = [64253033335, 51079096620, 904526161, 58177000111, 51200, 56017275, 227194, 224277, 50861, 51249]

# data load

In [5]:
data_base = pd.read_csv('new_allfit_preds_base.csv')
data_all_to_1 = pd.read_csv('new_allfit_all_to_1_all_preds2.csv')
data_all_to_0 = pd.read_csv('new_allfit_all_to_0_all_preds2.csv')
data_inverse = pd.read_csv('new_allfit_inverse_all_preds2.csv')

In [25]:
def mean_diff(top_bot_lst):
    ############################################### df processing
    for i in top_bot_lst:
        colnm = str(i)
        tmp1 = pd.Series(data_all_to_1[colnm], name = f'all_to_1_{i}') # all to 1
        tmp2 = pd.Series(data_all_to_0[colnm], name = f'all_to_0_{i}') # all to 0
        tmp3 = pd.Series(data_inverse[colnm], name = f'inverse_{i}') # inverse
        tmp4 = pd.Series(tmp1-tmp2, name = f'diff_{i}') # diff
        globals()['final_df_{}'.format(i)] = pd.concat([data_base, tmp1, tmp2, tmp3, tmp4], axis = 1)

In [12]:
top_bot = [rf_top10_lst, rf_bot10_lst, M1_entropy_top10_lst, M1_entropy_bot10_lst]

In [37]:
for i in range(len(top_bot)):
    mean_diff(top_bot[i])

In [40]:
for j in range(len(top_bot)):
    for i in top_bot[j]:
        f"item: {i}, mean: {np.mean(globals()['final_df_{}'.format(i)][f'diff_{i}'])}"
    "##########################################################################################"

'item: 51277, mean: 0.1328631993682848'

'item: 50912, mean: 0.06941548161747572'

'item: 51006, mean: 0.1086956901592233'

'item: 63323026201, mean: -0.10875725173592234'

'item: 50862, mean: 0.08114032732815533'

...

'item: 227194, mean: -0.13763709218317152'

'item: 224277, mean: -0.07514331092815534'

'item: 50861, mean: -0.09975139004919094'

'item: 51249, mean: -0.07495271450614886'

'##########################################################################################'

ImproperUseError: Argument Subscript(value=Name(id='top_bot', ctx=Load()), slice=Index(value=Constant(value=0, kind=None)), ctx=Load()) is not a variable or an attribute.